<a href="https://colab.research.google.com/github/myllanes/Introduction-to-Deep-Learning/blob/main/HW6_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Michael Yllanes
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from transformers import SwinForImageClassification, AutoImageProcessor
from tqdm import tqdm
import time
from datetime import timedelta

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
num_epochs = 5
batch_size = 32
learning_rate = 2e-5
image_size = 224
num_classes = 100
hidden_dim = 768  # hidden dimension

# Data preprocessing
processor = AutoImageProcessor.from_pretrained("microsoft/swin-small-patch4-window7-224") # Change with microsoft/swin-tiny-patch4-window7-224 or microsoft/swin-small-patch4-window7-224
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std)
])

# Load CIFAR-100
train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# Classifier
class classifierH(nn.Module):
    def __init__(self, hidden_dim, num_classes):
        super().__init__()

        self.conv = nn.Conv2d(hidden_dim, hidden_dim//2, kernel_size=1)
        self.bn = nn.BatchNorm2d(hidden_dim//2)
        self.relu = nn.ReLU()
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(hidden_dim//2, num_classes)
        # Zero-padding
        self.pad = nn.ZeroPad2d((0, 0, 0, max(0, 100 - num_classes)))

    def forward(self, x):
        x = x.unsqueeze(-1).unsqueeze(-1)
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.pool(x).flatten(1)
        x = self.fc(x)
        return self.pad(x)

# Load model with custom head
model = SwinForImageClassification.from_pretrained(
    "microsoft/swin-tiny-patch4-window7-224",
    num_labels=num_classes,
    ignore_mismatched_sizes=True
).to(device)

# Replace classifier
model.classifier = classifierH(hidden_dim, num_classes).to(device)

# Freeze backbone, train only head for fine tune
for param in model.parameters():
    param.requires_grad = False
for param in model.classifier.parameters():
    param.requires_grad = True

print("\nTrainable Parameters:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"\t{name}")

# Optimizer and loss
optimizer = torch.optim.AdamW(model.classifier.parameters(), lr=learning_rate, weight_decay=0.01)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

def train_epoch(epoch):
    model.train()
    epoch_start = time.time()
    total_loss = 0
    correct = 0
    total = 0

    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images).logits
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Metrics
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        progress_bar.set_postfix({
            'loss': total_loss/(progress_bar.n+1),
            'acc': f'{100.*correct/total:.1f}%'
        })

    epoch_time = time.time() - epoch_start
    epoch_acc = 100 * correct / total
    avg_loss = total_loss / len(train_loader)

    scheduler.step()

    return epoch_time, epoch_acc, avg_loss

def evaluate():
    model.eval()
    start_time = time.time()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc='Testing'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_time = time.time() - start_time
    accuracy = 100 * correct / total
    return test_time, accuracy

if __name__ == '__main__':
    print("=== Training with Conv-Padded Classifier Head ===")
    print(f"Using device: {device}\n")

    # Training loop
    train_times = []
    train_accs = []

    for epoch in range(num_epochs):
        epoch_time, epoch_acc, epoch_loss = train_epoch(epoch)
        train_times.append(epoch_time)
        train_accs.append(epoch_acc)

        print(f"\nEpoch {epoch+1} Summary:")
        print(f"Time: {timedelta(seconds=epoch_time)}")
        print(f"Train Accuracy: {epoch_acc:.2f}%")
        print(f"Avg Loss: {epoch_loss:.4f}\n")

    # Final evaluation
    test_time, test_acc = evaluate()

    # Results summary
    print("\n=== Final Results ===")
    print(f"Total Training Time: {timedelta(seconds=sum(train_times))}")
    print(f"Average Epoch Time: {timedelta(seconds=sum(train_times)/num_epochs)}")
    print("\nTraining Accuracies by Epoch:")
    for i, acc in enumerate(train_accs):
        print(f"Epoch {i+1}: {acc:.2f}% (Time: {timedelta(seconds=train_times[i])})")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
100%|██████████| 169M/169M [00:03<00:00, 42.9MB/s]


config.json:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/113M [00:00<?, ?B/s]

...


Epoch 4 Summary:
Time: 0:02:15.518480
Train Accuracy: 67.88%
Avg Loss: 1.5859



Epoch 5/5: 100%|██████████| 1563/1563 [02:15<00:00, 11.53it/s, loss=1.51, acc=68.7%]



Epoch 5 Summary:
Time: 0:02:15.531203
Train Accuracy: 68.73%
Avg Loss: 1.5089



Testing: 100%|██████████| 313/313 [00:26<00:00, 11.63it/s]


=== Final Results ===
Total Training Time: 0:11:15.388637
Average Epoch Time: 0:02:15.077727

Training Accuracies by Epoch:
Epoch 1: 35.81% (Time: 0:02:13.410698)
Epoch 2: 60.53% (Time: 0:02:15.380927)
Epoch 3: 65.60% (Time: 0:02:15.547329)
Epoch 4: 67.88% (Time: 0:02:15.518480)
Epoch 5: 68.73% (Time: 0:02:15.531203)
